**Part 4**

GRPO Loss

Improvements:
* Added Log Counter as train function parameter

Attempted the Following:
* Adding Gradient Clipping
* Tried adjusting the reward logic
* Added Entropy penalty to Loss calculation

Look into:
* detatching advantages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from tqdm import tqdm

In [6]:
# Define the neural network
class LogicNet(nn.Module):
    def __init__(self):
        super(LogicNet, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Input layer -> Hidden Layer
        self.fc2 = nn.Linear(4, 1)  # Hidden Layer -> Output Layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

    def get_action_and_or_log_prob(self, state, action=None):
        """Helper method to get action and its log_prob from logits"""
        logits = self.forward(state)    # Get the logits from a forward pass of the Policy Network
        # For a binary output (0 or 1), Bernoulli distribution is appropriate
        probs = torch.distributions.Bernoulli(logits=logits)

        if action is None:
            sampled_action = probs.sample() # Sample action based on current probabilities (returns 0 or 1)
            log_prob = probs.log_prob(sampled_action)   # Calculate the log of the probability the sampled action is chosen
            return sampled_action, log_prob
        else:
            log_prob = probs.log_prob(action)       # Returns the log of the probability the action is chosen
            return log_prob


In [7]:
# Define the environment
class LogicGateEnv:
    def __init__(self, gate="AND"):
        self.gate = gate
        self.data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
        self.targets = self.get_targets(gate)

    def get_targets(self, gate:str):
        if gate == "AND":
            return torch.tensor([[0], [0], [0], [1]], dtype=torch.float32)
        elif gate == "OR":
            return torch.tensor([[0], [1], [1], [1]], dtype=torch.float32)
        elif gate == "XOR":
            return torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)
        elif gate == "XNOR":
            return torch.tensor([[1], [0], [0], [1]], dtype=torch.float32)

    def step(self, input_idx: int, prediction):
        target = self.targets[input_idx].item()
        # Rounds up to 1 if it is >=.5 to get prediction; else 0
        reward = 1.0 if round(prediction) == target else -10.0
        return reward

In [ ]:
# Training loop
def train_logic_gate(gate="XOR", epochs=100, learning_rate=0.0001, batch_size=64, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.5, log_iterations=10):
    print(f"Training {gate} gate with {epochs} epochs, {learning_rate} learning rate, batch size {batch_size}, and KL beta {beta_kl}.")
    # Initialize Agent's Policy, Environment, parameter optimizer, and Total Correct Counter
    env = LogicGateEnv(gate)
    Policy_New = LogicNet()   # STEP 1 || CREATE π_new
    optimizer = optim.Adam(Policy_New.parameters(), lr=learning_rate)
    overall_num_correct = 0.0
    # STEP 2 || FOR I ITERATION STEPS OMITTED
    # STEP 3 || CREATE REFERENCE MODEL OMITTED

    for epoch in range(epochs):     # STEP 4 || FOR M ITERATION STEPS
        rewards_batch = []  # will be a list of floats
        inputs_batch = []   # will be a list of tensors
        targets_batch = []  # will be a list of tensors
        # log_prob_batch =[]  # will be a list of tensors

        # STEP 5 || Sample a batch D_b from D --> OMITTED 
        # STEP 6 || Update the old policy model π_old <- π_new
        Policy_Old = LogicNet()
        Policy_Old.load_state_dict(Policy_New.state_dict())
        Policy_Old.eval()   # Prevent Gradient tracking

        # --- STEP 7 || Collect a Batch of Experiences ---
        # Loop agent prediction, recording important values to lists:
        for i in range(batch_size):
            # Get model inputs and target
            idx = random.randint(0, 3)
            inputs = env.data[idx]
            target = env.targets[idx]

            # Get model prediction and log_prob of that prediction using the old policy
            with torch.no_grad(): # No need to track gradients during data collection
                pred, log_prob  = Policy_Old.get_action_and_or_log_prob(state=inputs)   # returns tensors

            # Calculate reward
            reward = env.step(idx, pred.item())

            # Append to lists
            inputs_batch.append(inputs)
            rewards_batch.append(reward)
            targets_batch.append(target)
            # log_prob_batch.append(log_prob)

        # Convert collected batch lists into PyTorch tensors
        inputs_batch_tensor = torch.stack(inputs_batch)     # Shape: (batch_size, 3)
        targets_old_batch_tensor = torch.stack(targets_batch)   # Shape: (batch_size, 1)
        rewards_batch_tensor = torch.tensor(rewards_batch, dtype=torch.float32)     # Shape: (batch_size,)
        # log_prob_old_batch_tensor = torch.stack(log_prob_batch)   # Shape: (batch_size, 1)

        overall_num_correct += (rewards_batch_tensor==1.0).sum().item()  ### need to change
        # print(f"Number correct, this iteration: {(rewards_batch_tensor).sum().item()}")

        # STEP 8 || Calculate Discounted Rewards
        # Unsqueeze to ensure rewards_batch_t has the same shape as targets_batch_t for element-wise ops SHAPE:(1, batch_size)
        rewards_batch_t = rewards_batch_tensor.unsqueeze(1)

        # --- STEP 9 || START OF ADVANTAGE CALCULATION ---
        # Calculate the mean of the rewards in the current batch
        mean_reward = rewards_batch_tensor.mean()

        # Calculate the standard deviation of the rewards in the current batch
        # Add a small epsilon (1e-8) to prevent division by zero in case all rewards are identical
        std_reward = rewards_batch_tensor.std() + 1e-8

        # print(f"rewards_batch_t shape: {rewards_batch_t.shape} || mean_reward: {mean_reward}")
        # Calculate the advantage for each time step in the batch using your specified formula
        advantages_of_batch = (rewards_batch_t - mean_reward) / (std_reward)
        # --- END OF ADVANTAGE CALCULATION ---

        # Detach these to prevent gradients from flowing back into old_policy_net
        inputs_batch_tensor = inputs_batch_tensor.detach()
        targets_old_batch_tensor = targets_old_batch_tensor.detach()
        rewards_batch_tensor = rewards_batch_tensor.detach()
        # log_prob_old_batch_tensor = log_prob_old_batch_tensor.detach()
        advantages_of_batch = advantages_of_batch.detach()

        # Get log_probabilities for the collected 'targets' from the OLD policy
        # Detach these to prevent gradients from flowing back into old_net
        with torch.no_grad():
            old_logits = Policy_Old(inputs_batch_tensor)
            # Use the get_action_and_or_log_prob helper
            log_prob_old = Policy_Old.get_action_and_or_log_prob(inputs_batch_tensor, targets_old_batch_tensor).detach()
            q_dist = torch.distributions.Bernoulli(logits=old_logits.detach())


        # --- STEP 10 || GRPO Optimization ---
        for _ in tqdm(range(k_epochs), desc=f"Epoch {epoch+1}/{epochs} (Inner K-Epochs)", leave=False):
            new_policy_logits = Policy_New(inputs_batch_tensor)
            log_prob_new = Policy_New.get_action_and_or_log_prob(inputs_batch_tensor, targets_old_batch_tensor)


            # --- KL Divergence Calculation ---
            # Create Bernoulli distributions for new policy using logits
            p_dist = torch.distributions.Bernoulli(logits=new_policy_logits)

            # Calculate KL divergence per sample, then take the mean over the batch
            kl_div_per_sample = torch.distributions.kl.kl_divergence(p_dist, q_dist)
            kl_loss = kl_div_per_sample.mean() # Mean over the batch

            # print(f"log_prob_new: {log_prob_new}")
            # print(f"log_prob_old: {log_prob_old}")

            # Calculate the ratio of each Trajectory in the Group
            # r_t(0) = π_0(a_t|s_t) / π_0_old(a_t|s_t) = exp(log(π_0(a_t|s_t) - log(π_0_old(a_t|s_t)))
            ratio = torch.exp(log_prob_new - log_prob_old)

            # print(f"Ratio: {ratio}")

            surrogate_1 = ratio * advantages_of_batch
            surrogate_2 = torch.clamp(input=ratio, min= 1.0 - epsilon, max= 1.0 + epsilon) * advantages_of_batch

            # Combine clipped loss with KL penalty
            # Remember: we minimize the negative of the main objective, and add the KL term
            # Maximize: min(...) - beta * D_KL(...) => Minimize: -min(...) + beta * D_KL(...)
            policy_objective_term = -torch.min(surrogate_1, surrogate_2).mean()

            # print(f"policy_objective_term: {policy_objective_term}")
            
            # Calculate the entropy of the New Policy distribution
            entropy = p_dist.entropy().mean()
            # print(f"Entropy Monitor: {entropy_coeff * entropy}") # Entropy goes to 0
            loss = policy_objective_term + beta_kl * kl_loss - entropy_coeff * entropy# Add KL term with beta_kl weight

            # In GRPO, the objective function is typically designed to be maximized (e.g., maximizing the expected return). Since PyTorch optimizers are designed for minimization, the common practice is to minimize the negative of the objective function.

            # STEP 11 || Policy Updates
            optimizer.zero_grad()
            loss.backward()
            # --- ADDING GRADIENT CLIPPING HERE TO LIMIT PARAMETER UPDATES---
            # torch.nn.utils.clip_grad_norm_(Policy_New.parameters(), max_norm=max_grad_norm)
            # ----------------------------------
            optimizer.step()


        # LOG METRICS IF ENOUGH EPOCHS HAVE ELAPSED
        if epoch % log_iterations == 0:
            avg_reward = rewards_batch_tensor.mean().item()
            print(f"Epoch {epoch}: Loss = {loss.item()}, Avg Reward = {avg_reward:.4f}, Mean Advantage: {advantages_of_batch.mean().item()}")
            # Validation Step
            print("Validating the Model:")
            with torch.no_grad():
                for i in range(4):
                    logits = Policy_New(env.data[i])
                    pred = torch.round(torch.sigmoid(logits)).item()    # Use Greedy Policy
                    print(f"Input: {env.data[i].tolist()}, Logits: {logits}, Prediction: {pred}, Actual: {env.targets[i].item()}")

    print("Training completed.\n")
    print(f"Number of correct predictions: {overall_num_correct}/{epochs * batch_size}")
    print(f"Accuracy: {overall_num_correct/(epochs * batch_size)}%")

    print("\nTesting Trained Model:")
    for i in range(4):
        logits = Policy_New(env.data[i])
        pred = torch.round(torch.sigmoid(logits)).item()
        print(f"Input: {env.data[i].tolist()}, Prediction: {pred}, Actual: {env.targets[i].item()}")


In [ ]:
# Run training
train_logic_gate(gate="AND", epochs=100, learning_rate=0.0001, batch_size=64, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.5, log_iterations=10)

Training AND gate with 100 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.


Epoch 1/100 (Inner K-Epochs):   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 0: Loss = -0.3479645848274231, Avg Reward = -5.7031, Mean Advantage: 1.862645149230957e-09
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.0161]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([0.0522]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.0125]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([0.0496]), Prediction: 1.0, Actual: 1.0


Epoch 10: Loss = -0.34693479537963867, Avg Reward = -3.9844, Mean Advantage: -2.9802322387695312e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.0280]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([0.0830]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.0409]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([0.0977]), Prediction: 1.0, Actual: 1.0


Epoch 20: Loss = -0.3459464907646179, Avg Reward = -3.1250, Mean Advantage: -2.60770320892334e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.0448]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([0.1196]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.0740]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([0.1501]), Prediction: 1.0, Actual: 1.0


...

Epoch 60: Loss = -0.3283790051937103, Avg Reward = -3.6406, Mean Advantage: -1.1175870895385742e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.3112]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([0.6268]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.5116]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([0.8273]), Prediction: 1.0, Actual: 1.0


Epoch 70: Loss = -0.3183174729347229, Avg Reward = -5.1875, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.4593]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([0.8859]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.7442]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([1.1709]), Prediction: 1.0, Actual: 1.0


Epoch 80: Loss = -0.288735955953598, Avg Reward = -5.7031, Mean Advantage: 1.862645149230957e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.6380]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([1.2173]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([1.0299]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([1.6092]), Prediction: 1.0, Actual: 1.0


Epoch 90: Loss = -0.2419928014278412, Avg Reward = -6.5625, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([0.8836]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([1.6479]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([1.4245]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([2.1888]), Prediction: 1.0, Actual: 1.0


Training completed.

Number of correct predictions: 2952.0/6400
Accuracy: 0.46125%

Testing Trained Model:
Input: [0.0, 0.0], Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Prediction: 1.0, Actual: 1.0


In [13]:
train_logic_gate(gate="AND", epochs=1000, learning_rate=0.0001, batch_size=64, k_epochs=128, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5, entropy_coeff=0.5, log_iterations=100)

Training AND gate with 1000 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.


Epoch 0: Loss = -0.3463467061519623, Avg Reward = -4.1562, Mean Advantage: -2.9802322387695312e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-0.1245]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-0.0740]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-0.1057]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-0.0129]), Prediction: 0.0, Actual: 1.0


Epoch 100: Loss = -0.25097307562828064, Avg Reward = -3.4688, Mean Advantage: 2.2351741790771484e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-1.3223]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-1.5917]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-1.6389]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-1.9857]), Prediction: 0.0, Actual: 1.0


Epoch 200: Loss = -0.06561212241649628, Avg Reward = -1.2344, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-6.9779]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-10.6638]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-11.3443]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-15.5988]), Prediction: 0.0, Actual: 1.0


Epoch 300: Loss = -0.09521104395389557, Avg Reward = -2.9531, Mean Advantage: 1.1175870895385742e-08
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-15.5039]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-25.8845]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-26.7027]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-37.7966]), Prediction: 0.0, Actual: 1.0


...

Epoch 600: Loss = -0.08764027059078217, Avg Reward = -1.9219, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-44.1812]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-79.4544]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-80.1168]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-116.1210]), Prediction: 0.0, Actual: 1.0


Epoch 700: Loss = -0.0774502158164978, Avg Reward = -1.0625, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-54.3207]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-98.7136]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-99.3270]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-144.4509]), Prediction: 0.0, Actual: 1.0


Epoch 800: Loss = -0.08764027059078217, Avg Reward = -1.9219, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-64.7697]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-118.6539]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-119.2212]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-173.8364]), Prediction: 0.0, Actual: 1.0


Epoch 900: Loss = -0.09197536110877991, Avg Reward = -2.4375, Mean Advantage: 0.0
Validating the Model:
Input: [0.0, 0.0], Logits: tensor([-75.5170]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-139.2401]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([-139.7638]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([-204.2180]), Prediction: 0.0, Actual: 1.0


Training completed.

Number of correct predictions: 46392.0/64000
Accuracy: 0.724875%

Testing Trained Model:
Input: [0.0, 0.0], Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Prediction: 0.0, Actual: 0.0
Input: [1.0, 1.0], Prediction: 0.0, Actual: 1.0
